<div class="alert alert-block alert-warning">

# Explore Exercises

In [1]:
# Imports

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

from sklearn.model_selection import train_test_split

from wrangle import acquire_zillow, get_zillow_data, wrangle_zillow, split_train_val_test

from explore import plot_variable_pairs, plot_categorical_and_continuous_vars

import os

<div class="alert alert-block alert-success">

Do you work for this exercise in either a jupyter notebook named evaluate within your regression-exercises repository. By the end of this exercise you will also create a python script named evaluate, so the overall deliverables for this exercise are the python script and jupyter notebook both with the name evaluate within your regression-exercises repo.


<div class="alert alert-block alert-info">

1. Load your zillow dataset.


In [ ]:
df = get_zillow_data()
df = wrangle_zillow(df)

train, val, test = split_train_val_test(df)

train.shape, val.shape, test.shape

((1498164, 7), (321035, 7), (321036, 7))

In [ ]:
train.head()

,bedrooms,bathrooms,area,tax_value,year_built,taxamount,fips
1007466,3,2.0,1452,241012,1980,2974.81,6037
615622,3,2.0,1262,235630,1953,3029.22,6037
662507,3,1.0,1447,435123,1947,5226.67,6037
840040,3,2.0,1403,268770,1965,2849.62,6111
243798,3,2.0,1456,365000,1961,4582.48,6037


<div class="alert alert-block alert-info">

2. Fit a linear regression model (ordinary least squares) and compute yhat, predictions of taxvaluedollarcnt using only calculatedfinishedsqft. Here is some sample code to get you started:  from sklearn.linear_model import LinearRegression


<div class="alert alert-block alert-info">

3. Plot the residuals for the linear regression model that you made.


<div class="alert alert-block alert-info">

4. Calculate the sum of squared errors, explained sum of squares, total sum of squares, mean squared error, and root mean squared error for your model.


<div class="alert alert-block alert-info">

5. Calculate the sum of squared errors, mean squared error, and root mean squared error for the baseline model (i.e. a model that always predicts the average taxvaluedollarcnt amount).


<div class="alert alert-block alert-info">

6. Write python code that compares the sum of squared errors for your model against the sum of squared errors for the baseline model and outputs whether or not your model performs better than the baseline model.


<div class="alert alert-block alert-info">

7. What is the amount of variance explained in your model?


<div class="alert alert-block alert-info">

8. Is your model better than the baseline model?


<div class="alert alert-block alert-info">

9. Create a file named evaluate.py that contains the following functions.
    * plot_residuals(y, yhat): creates a residual plot
        * regression_errors(y, yhat): returns the following values:
        * sum of squared errors (SSE)
        * explained sum of squares (ESS)
        * total sum of squares (TSS)
        * mean squared error (MSE)
        * root mean squared error (RMSE)
    * baseline_mean_errors(y): computes the SSE, MSE, and RMSE for the baseline model
    * better_than_baseline(y, yhat): returns true if your model performs better than the baseline, otherwise false